In [3]:
# Import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [4]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Initialize the webdriver
count = 0
url_path = r"https://www.google.com/maps"
driver.get(url_path)

# initiate search
search_input = input("Enter the search key word/s for Google Maps : ")
xpath_searchbox = r'//*[@id="searchboxinput"]'
enter = driver.find_element(By.XPATH,xpath_searchbox)
enter.send_keys(search_input)
enter.send_keys(Keys.ENTER)

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "pane")))

<selenium.webdriver.remote.webelement.WebElement (session="59ee3622f19952027624c71624f2dcaf", element="f.9ACF1A8F82F7691A78EB0518BFB16FF5.d.F4FFA8E8EE54C83190022049F9612B38.e.42")>

In [5]:
def scroll_down(driver,pixels=500):
    arg = "window.scrollTo(0, );"
    arg = arg[:-3] + str(pixels) + arg[-3+1:]
    driver.execute_script(arg,"")
    time.sleep(2)

In [6]:
# get the height of the page
last_height = driver.execute_script("return document.body.scrollHeight")

# loop to scroll down the page
while True:
    # scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # wait for the page to load
    time.sleep(2)

    # calculate the new height of the page
    new_height = driver.execute_script("return document.body.scrollHeight")

    # check if the page height has changed
    if new_height == last_height:
        break

    # update the last height
    last_height = new_height

In [7]:
xpath_objects = r'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a'
odd = 3
names = []
#addresses = []
urls = []
status = []
address = []
wait = WebDriverWait(driver, 10)
screen_height = driver.execute_script("return window.screen.height;")
count = 0
pixels = 50
driver.maximize_window()
time.sleep(2)
while True:
    pixels += 2500
    scroll_down(driver,pixels)
    time.sleep(1)
    xpath_objects = r'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a'
    xpath_objects = xpath_objects[:-8] + str(odd) + xpath_objects[-8+1:]
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, xpath_objects)))
        driver.find_element(By.XPATH, xpath_objects).click()
    except:
        pixels += 2500
        scroll_down(driver,pixels)
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, xpath_objects)))
            driver.find_element(By.XPATH, xpath_objects).click()
        except:
            break
    scroll_down(driver)
    try:
        wait.until(EC.presence_of_element_located((By.XPATH,r'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[7]/div[3]/button/div[1]/div[2]/div[1]')))
        address.append(driver.find_element(By.XPATH,r'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[7]/div[3]/button/div[1]/div[2]/div[1]').text)
    except:
        address.append("N/A")
    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME,"aSftqf")))
        status.append(driver.find_element(By.CLASS_NAME,"aSftqf").text)
    except:
        status.append("Open")
    urls.append(driver.current_url)
    time.sleep(1)
    #driver.execute_script("window.history.go(-1)")
    wait.until(EC.presence_of_element_located((By.XPATH,r'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[1]/div/div/div[3]/span/button')))
    driver.find_element(By.XPATH,r'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[1]/div/div/div[3]/span/button').click()
    count += 1
    odd += 2

In [8]:
import pandas as pd
df = pd.DataFrame()
lats = []
longs = []
names = []
for url in urls:
    # Extract the substring after the "@" character
    coordinates = url.split("@")[1].split(",")
    # Extract the latitude and longitude
    latitude = float(coordinates[0])
    longitude = float(coordinates[1].split("z")[0])
    lats.append(latitude)
    longs.append(longitude)
    # to extract the names
    # Extract the substring after "place/" and before "/"
    name = url.split("place/")[1].split("/")[0]
    # Replace "+" with " "
    names.append(name.replace("+", " "))

df["Name"] = names
df["Status"] = status
df["Latitude"] = lats
df["Longitude"] = longs
df["Address"] = address
df["URL"] = urls
df

c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Name,Status,Latitude,Longitude,Address,URL


In [8]:
path = 'C:/Users/Nadeem/Desktop/PFA/Google_Maps_Data' 
path = path + "/" + search_input + ".csv"
df.to_csv(path)